In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive, files
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import data
application_df = pd.read_csv("https://raw.githubusercontent.com/ehalprin/Neural_Network_Charity_Analysis/main/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

**FURTHER REDUCE NUMBER OF APPLICATION TYPES**

In [4]:
# Bin application types - further reduce total # of application types (application count < 1000 rather than 500 as initially tested)
application_counts = application_df["APPLICATION_TYPE"].value_counts()
replace_application = application_counts[application_counts < 1000].index.tolist()
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [5]:
# Bin classification types
classification_counts = application_df["CLASSIFICATION"].value_counts()
replace_class = classification_counts[classification_counts < 1883].index.tolist()
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [6]:
# View finished dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

**REMOVE NOISY VARIABLE**

In [8]:
# Drop "Status" column
# application_df = application_df.drop(columns=['STATUS'])

In [9]:
# Encode categorical variable lists
apps_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
enc = OneHotEncoder(sparse=False)
encode_df = pd.DataFrame(enc.fit_transform(application_df[apps_cat]))
encode_df.columns = enc.get_feature_names_out(apps_cat)

# Merge encoded & original dataframe, & drop unnecessary columns
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(apps_cat, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**INCREASE # OF NODES AND ADD ANOTHER LAYER**

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

**CHANGE ACTIVATION TYPE OF HIDDEN LAYERS**

In [13]:
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="LeakyReLU"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="LeakyReLU"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="LeakyReLU"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                3690      
                                                                 
 dense_1 (Dense)             (None, 40)                3640      
                                                                 
 dense_2 (Dense)             (None, 10)                410       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

In [16]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 6s 6ms/step - loss: 0.5708 - accuracy: 0.7232
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5582 - accuracy: 0.7297
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5559 - accuracy: 0.7307
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7318
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7324
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7321
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7338
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7340
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7341
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7335

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5601 - accuracy: 0.7247 - 505ms/epoch - 2ms/step
Loss: 0.5600659847259521, Accuracy: 0.7246647477149963


In [18]:
# Export model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")
files.download('AlphabetSoupCharity_Optimization.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>